**This file allows you to download all the data from an influx DB on a webserver into .csv files.**

In [ ]:
import requests
import os

#Get URL
baseURL = input("Website URL: ")
if (URL[-1] != '/'):
    URL += '/'

#Get file path
filePath = input("Relative path to where you'd like to download the data: ")
if (filePath != '' && filePath[-1] != '/'):
    filePath += '/'

#Create folders for file path if they dont exist
os.makedirs(filePath, exist_ok=True)

#Get location information from server
infoResponse = requests.get(url = (baseURL + 'sensor/info'))
infoData = infoResponse.json()["locations"]

#Loop through each location
for location in infoData:
    #Create this node's .csv file
    file = open(filePath + "node" + location[0] + ".csv", "w+")
    
    #Write the node's name
    file.write(location[1] + '\n')
    
    #Get the type information for this node
    params = {'id': location[0]}
    types = (requests.get(url = (baseURL + 'sensor/type'), params = params)).json()["types"]
    
    #Write the types
    file.write("Time,Latitude,Longitude")
    for i in types:
        file.write(',' + i)
    file.write('\n')
    
    #Get the location history for this node
    allLocations = (requests.get(url = (baseURL + 'sensor/info'), params = params)).json()["locations"]
    
    #If there are sensors
    if (len(types) != 0):
        #Get the sensor data for every sensor
        data = []
        for i in types:
            params = {'id': location[0], 'type': i}
            currData = (requests.get(url = (baseURL + 'sensor/data'), params = params)).json()["data"]
            data.append(currData)

        curr = 0
        
        #Loop through every data point
        #Assumes all data points have data from every sensor
        for i in range(len(data[0])):
            #Write the time
            time = int(data[0][i][0]) // 1000000
            file.write(str(time) + ', ')
            
            #If this data point has a location
            if (int(allLocations[curr][0])//1000000) == time:
                #Write the location
                file.write(allLocations[curr][1])
                if (curr != len(allLocations)-1):
                    curr += 1
            else:
                file.write(', ')
            
            #Write the sensor data
            for j in range(len(data)):
                file.write(', ' + data[j][i][1])
            
            file.write('\n')
    
    #If it's only location data
    else:
        #Write the locations
        for i in allLocations:
            file.write(i[0] + ', ' + i[1] + '\n')
    
    file.close()

print("All done")

#URL for current setup: https://www.cas.mcmaster.ca/ollie